In [1]:
import numpy as np
import pandas as pd
import seaborn as sns
%matplotlib inline
import folium
from sklearn.preprocessing import LabelEncoder
from geopy.geocoders import Yandex
from geopy.distance import vincenty

In [2]:
train = pd.read_csv('data/train_set.csv')
test = pd.read_csv('data/test_set.csv')

train.rename(columns={ 'pos_adress_lat' : 'pos_address_lat', 
                       'pos_adress_lon' : 'pos_address_lon'}, inplace=True)

D:\Miniconda3\lib\site-packages\IPython\core\interactiveshell.py:2698: DtypeWarning: Columns (1,11) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)
D:\Miniconda3\lib\site-packages\IPython\core\interactiveshell.py:2698: DtypeWarning: Columns (1,8,9) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [3]:
train[:2]

,amount,atm_address,atm_address_lat,atm_address_lon,city,country,currency,customer_id,home_add_lat,home_add_lon,mcc,pos_address,pos_address_lat,pos_address_lon,terminal_id,transaction_date,work_add_lat,work_add_lon
0,2.884034,NaN,NaN,NaN,ST PETERSBURG,RUS,643.0,0dc0137d280a2a82d2dc89282450ff1b,59.851,30.232,5261,NaN,59.844072,30.179153,11606fde0c814ce78e0d726e39a0a5ee,2017-07-15,59.847,30.177
1,2.775633,NaN,NaN,NaN,ST PETERSBURG,RUS,643.0,0dc0137d280a2a82d2dc89282450ff1b,59.851,30.232,5261,NaN,59.844072,30.179153,e9647a5e1eacfb06713b6af755ccc595,2017-10-27,59.847,30.177


In [7]:
train1 = train[['atm_address', 'customer_id', 'pos_address', 'transaction_date']]

In [9]:
train1[:2]

,atm_address,customer_id,pos_address,transaction_date
0,NaN,0dc0137d280a2a82d2dc89282450ff1b,NaN,2017-07-15
1,NaN,0dc0137d280a2a82d2dc89282450ff1b,NaN,2017-10-27


In [6]:
train1[train1['customer_id'] == '0dc0137d280a2a82d2dc89282450ff1b']

,atm_address,customer_id,pos_address,transaction_date
0,NaN,0dc0137d280a2a82d2dc89282450ff1b,NaN,2017-07-15
1,NaN,0dc0137d280a2a82d2dc89282450ff1b,NaN,2017-10-27
2,NaN,0dc0137d280a2a82d2dc89282450ff1b,"PR.MARSHALA ZHUKOVA,31St Petersburg190000 7...",2017-10-03
3,NaN,0dc0137d280a2a82d2dc89282450ff1b,NaN,2017-09-09
4,NaN,0dc0137d280a2a82d2dc89282450ff1b,NaN,2017-07-06
5,NaN,0dc0137d280a2a82d2dc89282450ff1b,NaN,2017-08-23
6,NaN,0dc0137d280a2a82d2dc89282450ff1b,NaN,2017-07-29
7,NaN,0dc0137d280a2a82d2dc89282450ff1b,NaN,2017-07-15
8,NaN,0dc0137d280a2a82d2dc89282450ff1b,NaN,2017-08-18
9,NaN,0dc0137d280a2a82d2dc89282450ff1b,NaN,2017-05-13


In [11]:
train1

,atm_address,customer_id,pos_address,transaction_date
0,NaN,0dc0137d280a2a82d2dc89282450ff1b,NaN,2017-07-15
1,NaN,0dc0137d280a2a82d2dc89282450ff1b,NaN,2017-10-27
2,NaN,0dc0137d280a2a82d2dc89282450ff1b,"PR.MARSHALA ZHUKOVA,31St Petersburg190000 7...",2017-10-03
3,NaN,0dc0137d280a2a82d2dc89282450ff1b,NaN,2017-09-09
4,NaN,0dc0137d280a2a82d2dc89282450ff1b,NaN,2017-07-06
5,NaN,0dc0137d280a2a82d2dc89282450ff1b,NaN,2017-08-23
6,NaN,0dc0137d280a2a82d2dc89282450ff1b,NaN,2017-07-29
7,NaN,0dc0137d280a2a82d2dc89282450ff1b,NaN,2017-07-15
8,NaN,0dc0137d280a2a82d2dc89282450ff1b,NaN,2017-08-18
9,NaN,0dc0137d280a2a82d2dc89282450ff1b,NaN,2017-05-13
